# ETL BBDD PETICIDAS_UE
El objetivo de esta ETL es **extraer** de la api que dispone la Unión Europea  https://data.europa.eu/data/datasets/pesticides?locale=en para **transformar** la BBDD para luego **cargar** y usar este dataset transformado para la unión de tablas

## 1. Importamos librerias 

In [1]:
import requests
import pandas as pd
import ast

## 2. Lectura de datos y estructuras de dataframes

In [2]:
# Extrción de datos de la API de la Unión Europea sobre pesticidas
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
    'Accept': 'application/json',
    'Referer': 'https://api.datalake.sante.service.ec.europa.eu'
}

url = "https://api.datalake.sante.service.ec.europa.eu/sante/pesticides/active_substances?format=json&api-version=v2.0"

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    pesticidas = pd.json_normalize(data['value'])
else:
    print("Error:", response.status_code, response.text[:200])


In [3]:
pesticidas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 46 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   substance_id                       100 non-null    int64  
 1   substance_name                     100 non-null    object 
 2   as_cas_number                      100 non-null    object 
 3   substance_status                   100 non-null    object 
 4   approval_date                      64 non-null     object 
 5   expiry_date                        64 non-null     object 
 6   risk_assessment                    63 non-null     object 
 7   substance_category                 72 non-null     object 
 8   as_is_group                        100 non-null    object 
 9   as_micro_org                       100 non-null    object 
 10  rms                                59 non-null     object 
 11  corms                              39 non-null     object 


## 3. ETL

In [4]:
# Consultamos las columnas que nos interesan
pesticidas =pesticidas [['substance_name', 'as_cas_number']]
pesticidas.head()

,substance_name,as_cas_number
0,(E)-11-Tetradecen-1-yl acetate (SCLP Acetates),33189-72-9
1,Iminoctadine,13516-27-3
2,Silthiofam,175217-20-6
3,Endosulfan,115-29-7
4,"(E,Z)-7,9-Dodecadien-1-yl acetate (SCLP Acetates)",54364-62-4


Consultamos los valores únicos de ambas variables

In [5]:
pesticidas['as_cas_number'].nunique(dropna=True)

80

In [6]:
pesticidas[pesticidas['as_cas_number'].str.contains(r'[,;/]', regex= True, na=False)].count()

substance_name    0
as_cas_number     0
dtype: int64

In [7]:
# Nos quedamos con las variables que nos interesan
pesticidas = pesticidas[['substance_name', 'as_cas_number']]

# Renombramos las columnas
pesticidas.rename(columns={'substance_name': 'Name_Pesticida', 'as_cas_number': 'CAS Number'}, inplace=True)

In [8]:
# Exportamos el DataFrame 
pesticidas.to_parquet("../../data/processed/notebooks/pesticidas.parquet", index=False)